In [7]:
!pip install -U transformers==4.40.2
!pip install -U trl==0.8.6
!pip install -U accelerate
!pip install -U peft


   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   ---------- ----------------------------- 2.4/9.0 MB 13.4 MB/s eta 0:00:01
   ------------------------- -------------- 5.8/9.0 MB 16.0 MB/s eta 0:00:01
   ---------------------------------------- 9.0/9.0 MB 16.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 17.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.0
    Uninstalling transformers-4.55.0:
      Successfully uninstalled transformers-4.55.0


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.0.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.2 which is incompatible.
trl 0.21.0 requires transformers>=4.55.0, but you have transformers 4.40.2 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\IQBAL SINGH\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Attempting uninstall: trl
    Found existing installation: trl 0.21.0
    Uninstalling trl-0.21.0:
      Successfully uninstalled trl-0.21.0



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\IQBAL SINGH\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\IQBAL SINGH\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\IQBAL SINGH\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
import torch
import os

# List all Parquet files (assuming they're in ./data/)
data_dir = "./data"
parquet_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".parquet")]

# Load dataset
raw_dataset = load_dataset(
    "parquet",
    data_files={"train": parquet_files},
    split="train"
)

# Display sample
print(raw_dataset[0])

# Filter valid rows (some may be missing fields)
dataset = raw_dataset.filter(lambda x: x.get("code") is not None and x.get("docstring") is not None)

# Concatenate docstring + code as input, use dummy label for now
dataset = dataset.map(lambda x: {
    "text": f"{x['docstring']} </s> {x['code']}",
    "label": 0  # If you have actual labels, use them here
})

# Load tokenizer and model
model_ckpt = "mistralai/Mistral-7B-v0.1"  # or any smaller model like "distilbert-base-uncased" if running locally
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True, trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=1, trust_remote_code=True)

# Tokenize dataset
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)

# Split into train/valid
split = tokenized_dataset.train_test_split(test_size=0.1)
train_ds = split["train"]
eval_ds = split["test"]

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=1,
    logging_dir="./logs",
    logging_steps=10,
    remove_unused_columns=False,
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer)
)

# Train
trainer.train()

# Save model
trainer.save_model("./trained_model")
print(" Training complete. Model saved to './trained_model'.")


FileNotFoundError: [WinError 3] The system cannot find the path specified: './data'

In [6]:
!pip install transformers.utils.import_utils



ERROR: Could not find a version that satisfies the requirement transformers.utils.import_utils (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\IQBAL SINGH\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for transformers.utils.import_utils
